In [27]:
import requests
import pandas as pd
from datetime import datetime
import os

In [40]:
df_lines = pd.read_csv('referentiel-des-lignes.csv', sep=';')
stops_data = pd.read_csv('arrets.csv', sep=';')
delays_metro = pd.read_csv("metro_delays.csv")
delays_rer = pd.read_csv("rail_delays.csv")
metro_incident = pd.read_csv("metro_line_reports.csv")
rer_incident = pd.read_csv("rer_line_reports.csv")

In [29]:
line_refs = df_lines[(~df_lines['TransportSubmode'].isin(['suburbanRailway', 'regionalRail', 'railShuttle']))]
line_refs = line_refs[['ID_Line', 'TransportMode', 'Name_Line']]
line_refs = line_refs.sort_values(by='ID_Line')

In [30]:
stops_data = stops_data[stops_data['ArRType'].isin(['metro', 'rail'])]
stops_data = stops_data.sort_values(by=['ArRType', 'ArRId'])
stops_data = stops_data[['ArRId', 'ArRName', 'ArRType', 'ArRTown']]

In [31]:
metro_incident['ref'] = metro_incident['ref'].str.replace('stop_point:IDFM:', '', regex=False)
metro_incident['ref'] = metro_incident['ref'].str.replace('line:IDFM:', '', regex=False)
metro_incident['ref'] = metro_incident['ref'].str.replace('stop_area:IDFM:', '', regex=False)

In [41]:
rer_incident['ref'] = rer_incident['ref'].str.replace('stop_point:IDFM:', '', regex=False)
rer_incident['ref'] = rer_incident['ref'].str.replace('line:IDFM:', '', regex=False)
rer_incident['ref'] = rer_incident['ref'].str.replace('stop_area:IDFM:', '', regex=False)

In [32]:
stops_filter = [22086, 463013, 22136, 462993, 21964, 462969, 22125, 463113, 41295, 473921, 473993, 41354, 474060, 474061]
lines_filter = ["C01371", "C01372", "C01374" ," C01382", "C01742", "C01743", "C01727", "C0172"]

In [33]:
filtered_stops = stops_data[stops_data['ArRId'].isin(stops_filter)]

In [34]:
filtered_line = metro_incident[
    metro_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

In [42]:
filtered_line_rer = rer_incident[
    rer_incident['ref'].isin(map(str, stops_filter + lines_filter))
]


In [35]:
merged_metro = pd.merge(delays_metro, filtered_stops, left_on='stop_reference', right_on='ArRId')
merged_metro.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode'], axis=1, inplace=True) #inplace allows to actually modify the original datset without having to reassing with "="

In [36]:
merged_rer = pd.merge(delays_rer, filtered_stops, left_on='stop_reference', right_on='ArRId')
merged_rer.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode'], axis=1, inplace=True)